# 第8章 値としてのIO

In [1]:
case class MeetingTime(startHour: Int, endHour: Int)


defined class MeetingTime

In [2]:
// 副作用を持つAPI呼び出し
def calendarEntriesApiCall(name: String): List[MeetingTime] = ???
// import ch_08SchedulingMeetings.calendarEntriesApiCall
// jupyter notebook上ではimportできないのでsbt console上で実行する必要がある


defined function calendarEntriesApiCall

In [3]:
// 副作用を持つAPI呼び出し
def createMeetingApiCall(names: List[String], meetingTime: MeetingTime): Unit = ???
// import ch_08SchedulingMeetings.createMeetingApiCall
// jupyter notebook上ではimportできないのでsbt console上で実行する必要がある


defined function createMeetingApiCall

In [4]:
import $ivy.`org.typelevel::cats-effect:3.3.1`
import cats.effect._

def calendarEntries(name: String): IO[List[MeetingTime]] = {
  IO.delay(calendarEntriesApiCall(name));
}


import $ivy.$                                 

import cats.effect._


defined function calendarEntries

In [5]:
def scheduledMeetings(person1: String, person2: String): IO[List[MeetingTime]] = {
  for {
    person1Entries <- calendarEntries(person1)
    person2Entries <- calendarEntries(person2)
  } yield person1Entries.appendedAll(person2Entries)
}


defined function scheduledMeetings

In [6]:
def meetingsOverlap(meeting1: MeetingTime, meeting2: MeetingTime): Boolean = {
  meeting1.endHour > meeting2.startHour && meeting2.endHour > meeting1.startHour
}


defined function meetingsOverlap

In [7]:
def possibleMeetings(existingMeetings: List[MeetingTime], startHour: Int, endHour: Int, lengthHours: Int): List[MeetingTime] = {
  val slots = List
      .range(startHour, endHour - lengthHours + 1)
      .map(startHour => MeetingTime(startHour, startHour + lengthHours))

  slots.filter(slot =>
    existingMeetings.forall(meeting => !meetingsOverlap(meeting, slot)))
}


defined function possibleMeetings

In [8]:
def schedule(person1: String, person2: String, lengthHours: Int): IO[Option[MeetingTime]] = {
  for {
    existingMeetings <- scheduledMeetings(person1, person2)
    meetings = possibleMeetings(existingMeetings, 8, 16, lengthHours)
  } yield meetings.headOption
}


defined function schedule

## 8.27 実習: IO型の値によるリカバリー

In [9]:
import cats.implicits._

def castTheDie(): Int = ???
def drawAPointCard(): Int = ???

// 1
IO.delay(castTheDie()).orElse(IO.delay(drawAPointCard()))

// 2
IO.delay(drawAPointCard()).orElse(IO.delay(castTheDie()))

// 3
IO.delay(castTheDie())
  .orElse(IO.delay(drawAPointCard()))
  .orElse(IO.pure(0))

// 4
for {
  die <- IO.delay(castTheDie()).orElse(IO.pure(0))
  card <- IO.delay(drawAPointCard()).orElse(IO.pure(0))
} yield die + card

// 5
(for {
  card <- IO.delay(drawAPointCard())
  die1 <- IO.delay(castTheDie())
  die2 <- IO.delay(castTheDie())
} yield card + die1 + die2).orElse(IO.pure(0))


import cats.implicits._


defined function castTheDie
defined function drawAPointCard
res9_3: IO[Int] = HandleErrorWith(
  ioa = Delay(
    thunk = ammonite.$sess.cell9$Helper$$Lambda$3618/0x000000080168c258@11487d23,
    event = cats.effect.tracing.TracingEvent$StackTrace
  ),
  f = cats.syntax.ApplicativeErrorOps$$$Lambda$3621/0x000000080168d1a8@1fa516d5,
  event = cats.effect.tracing.TracingEvent$StackTrace
)
res9_4: IO[Int] = HandleErrorWith(
  ioa = Delay(
    thunk = ammonite.$sess.cell9$Helper$$Lambda$3622/0x000000080168d788@3fa306b8,
    event = cats.effect.tracing.TracingEvent$StackTrace
  ),
  f = cats.syntax.ApplicativeErrorOps$$$Lambda$3621/0x000000080168d1a8@688879c4,
  event = cats.effect.tracing.TracingEvent$StackTrace
)
res9_5: IO[Int] = HandleErrorWith(
  ioa = HandleErrorWith(
    ioa = Delay(
      thunk = ammonite.$sess.cell9$Helper$$Lambda$3624/0x000000080168dcf8@340021aa,
      event = cats.effect.tracing.TracingEvent$StackTrace
    ),
    f = cats.syntax.Applicat

## 8.28 潜在的な失敗にはどこで対処する？

In [9]:
// 1
// def calendarEntries(name: String): IO[List[MeetingTime]] = {
//   IO.delay(calendarEntriesApiCall(name))
//     .orElse(IO.delay(calendarEntriesApiCall(name)))
//     .orElse(IO.pure(List.empty))
// }

// 2
// def scheduledMeetings(person1: String, person2: String): IO[List[MeetingTime]] = {
//   for {
//     person1Entries <- calendarEntries(person1)
//                         .orElse(calendarEntries(person1))
//                         .orElse(IO.pure(List.empty))
//     person2Entries <- calendarEntries(person2)
//                         .orElse(calendarEntries(person2))
//                         .orElse(IO.pure(List.empty))
//   } yield person1Entries.appendedAll(person2Entries)
// }

// 3
def schedule(person1: String, person2: String, lengthHours: Int): IO[Option[MeetingTime]] = {
  for {
    existingMeetings <- scheduledMeetings(person1, person2)
        .orElse(scheduledMeetings(person1, person2))
        .orElse(IO.pure(List.empty))
    meetings = possibleMeetings(existingMeetings, 8, 16, lengthHours)
    possibleMeeting = meetings.headOption
    _ <- possibleMeeting match {
      case Some(meeting) => createMeeting(List(person1, person2), meeting)
      case None          => IO.unit
    }
  } yield meetings.headOption
}


-- [E006] Not Found Error: cell10.sc:9:28 --------------------------------------
9 |      case Some(meeting) => createMeeting(List(person1, person2), meeting)
  |                            ^^^^^^^^^^^^^
  |                            Not found: createMeeting
  |
  | longer explanation available when compiling with `-explain`
Compilation Failed

## 8.38 実習: 関数型シグネチャで直観を働かせる

In [29]:
def ex1[A, B](x: List[A], y: A): List[A] = x.appended(y)
def ex2[A, B](x: List[A], f: A => B): List[B] = x.map(f)

def f01[A, B](x: IO[A], f: A => B): IO[B] = x.map(f)
def f02[A](x: IO[IO[A]]): IO[A] = x.flatten
def f03[A, B](x: IO[A], f: A => IO[B]): IO[B] = x.flatMap(f)
def f04[A](x: A): IO[A] = IO.pure(x)
def f05[A](impureAction: () => A): IO[A] = IO.delay(impureAction())
def f06[A](x: IO[A], alternative: IO[A]): IO[A] = x.orElse(alternative)
def f07[A](x: List[IO[A]]): IO[List[A]] = x.sequence
def f08[A](x: Option[IO[A]]): IO[Option[A]] = x.sequence
def f09[A, B](x: List[A], y: List[A]): List[A] = x.appendedAll(y)
def f10[A](x: List[A], f: A => Boolean): List[A] = x.filter(f)
def f11[A](x: List[A], zero: A, f: (A, A) => A): A = x.foldLeft(zero)(f)
def f12[A](x: List[List[A]]): List[A] = x.flatten
def f13[A, B](x: List[A], f: A => List[B]): List[B] = x.flatMap(f)
def f14[A](x: List[A], f: A => Boolean): Boolean = x.exists(f)
def f15[A, B](x: Set[A], f: A => B): Set[B] = x.map(f)
def f16[A](x: Set[A], f: A => Boolean): Set[A] = x.filter(f)
def f17[A](x: Set[A], zero: A, f: (A, A) => A): A = x.foldLeft(zero)(f)
def f18[A](x: Set[Set[A]]): Set[A] = x.flatten
def f19[A, B](x: Set[A], f: A => Set[B]): Set[B] = x.flatMap(f)
def f20[A](x: Set[A], f: A => Boolean): Boolean = x.forall(f)
def f21[A, B](x: Option[A], f: A => B): Option[B] = x.map(f)
def f22[A](x: Option[A], f: A => Boolean): Option[A] = x.filter(f)
def f23[A](x: Option[A], zero: A, f: (A, A) => A): A = x.foldLeft(zero)(f)
def f24[A](x: Option[Option[A]]): Option[A] = x.flatten
def f25[A, B](x: Option[A], f: A => Option[B]): Option[B] = x.flatMap(f)
def f26[A](x: Option[A], f: A => Boolean): Boolean = x.exists(f)
def f27(x: String): Option[Int] = x.toIntOption
def f28[A](x: Option[A], alternative: Option[A]): Option[A] = x.orElse(alternative)
def f29[A, B](x: Option[A], y: B): Either[B, A] = x.toRight(y)
def f30[A, B](x: Option[A], y: B): Either[A, B] = x.toLeft(y)
def f31[A](x: List[Option[A]]): Option[List[A]] = x.sequence
def f32[A, B, C](x: Either[A, B], f: B => C): Either[A, C] = x.map(f)
def f33[A, B, C](x: Either[A, B], zero: C, f: (C, B) => C): C = x.foldLeft(zero)(f)
def f34[A, B](x: Either[A, Either[A, B]]): Either[A, B] = x.flatten
def f35[A ,B, C](x: Either[A, B], f: B => Either[A, C]): Either[A, C] = x.flatMap(f)
def f36[A, B](x: Either[A, B], f: B => Boolean): Boolean = x.exists(f)
def f37[A, B](x: Either[A, B], alternative: Either[A, B]): Either[A, B] = x.orElse(alternative)
def f38[A, B](x: Either[A, B]): Option[B] = x.toOption
def f39[A, B](x: List[Either[A, B]]): Either[A, List[B]] = x.sequence
def f40[A, B](x: Either[A, List[B]]): List[Either[A, B]] = x.sequence


defined function ex1
defined function ex2
defined function f01
defined function f02
defined function f03
defined function f04
defined function f05
defined function f06
defined function f07
defined function f08
defined function f09
defined function f10
defined function f11
defined function f12
defined function f13
defined function f14
defined function f15
defined function f16
defined function f17
defined function f18
defined function f19
defined function f20
defined function f21
defined function f22
defined function f23
defined function f24
defined function f25
defined function f26
defined function f27
defined function f28
defined function f29
defined function f30
defined function f31
defined function f32
defined function f33
defined function f34
defined function f35
defined function f36
defined function f37
defined function f38
defined function f39
defined function f40